<a href="https://colab.research.google.com/github/xanasa14/Crypto/blob/main/Bitcoin_predictor2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Sentiment Analisys 

In [14]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

finwiz_url = 'https://finviz.com/quote.ashx?t='

In [15]:
tickers = ['COIN']
news_tables = {}

for ticker in tickers:
  url = finwiz_url + ticker
  req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
  response = urlopen(req)    
  # Read the contents of the file into 'html'
  html = BeautifulSoup(response)
  # Find 'news-table' in the Soup and load it into 'news_table'
  news_table = html.find(id='news-table')
  # Add the table to our dictionary
  news_tables[ticker] = news_table


In [16]:
# Read one single day of headlines for 'AMZN' 
amzn = news_tables['COIN']
# Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
amzn_tr = amzn.findAll('tr')

for i, table_row in enumerate(amzn_tr):
    # Read the text of the element 'a' into 'link_text'
    a_text = table_row.a.text
    # Read the text of the element 'td' into 'data_text'
    td_text = table_row.td.text
    # Print the contents of 'link_text' and 'data_text' 
    print(a_text)
    print(td_text)
    # Exit after printing 4 rows of data
    if i == 3:
        break

ARK Invest Stocks To Buy And Watch: 6 Stocks That Cathie Wood's ARK ETFs Own; Coinbase, Square, Tesla Race Higher
Jun-14-21 04:44PM  
Is Coinbase Stock A Buy Right Now As Bitcoin Surges Above $41,000?
04:42PM  
This Small 401(k) Provider and Coinbase Will Offer Cryptocurrencies in Retirement Plans. What to Know.
04:37PM  
What you need to know to start investing in cryptocurrency right now
11:05AM  
